In [2]:
import sys
import os
from pathlib import Path
project_root = os.path.join('/home/shashank/research/paper_tagging/Paper_list_tagger')
os.chdir(project_root)
%pwd 

'/home/shashank/research/paper_tagging/Paper_list_tagger'

In [24]:
import pandas as pd
import arxiv
import requests
from bs4 import BeautifulSoup
import time
import openvpn_api


In [4]:
# read the data
file_name = 'USC Robotic Embedded Systems Lab Content - Publications (use this).csv'
file_path = os.path.join(project_root, 'data', file_name)
df = pd.read_csv(file_path)

In [ ]:
# add an Abstract column to if it doesn't exist
if 'Abstract' not in df.columns:
    df['Abstract'] = None

Index(['Date', 'Authors', 'Type', 'Tags', 'Include?', 'Title', 'Details',
       'Bibtex ID', 'PDF ', 'Video', 'ArXiv', 'Codes', 'Slides', 'Poster',
       'Twitter', 'Linkedin', 'News', 'Source-News', 'Blog', 'Website',
       'Other', 'Source-other'],
      dtype='object')

In [29]:
def get_google_scholar_abstracts(query, num_results=10):
    base_url = "https://scholar.google.com/scholar"

    # Define user-agent headers to send with HTTP requests, use chrome
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    }

    params = {
        "q": query,
        "num": num_results
    }

    # Send an HTTP GET request with headers
    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        abstracts = []
        for result in soup.find_all('div', class_='gs_ri'):
            title = result.find('h3', class_='gs_rt').text
            abstract = result.find('div', class_='gs_rs')
            if abstract:
                abstract = abstract.text
            else:
                abstract = "No abstract available"
            abstracts.append((title, abstract))

        return abstracts, response
    else:
        print("Error: Unable to retrieve search results")
        return None, response

# # Example usage:
query = df['Title'][1]
abstracts, response = get_google_scholar_abstracts(query, num_results=5)
# for i, (title, abstract) in enumerate(abstracts, start=1):
#     print(f"Paper {i} - Title: {title}\nAbstract: {abstract}\n")


Error: Unable to retrieve search results


In [31]:
openvpn_api.VPN('localhost', 7505).connect()

ConnectError: [Errno 111] Connection refused

In [ ]:
# add an abstract column, using the get_google_scholar_abstracts function
df["Abstract"] = None
for i in range(len(df)):
    print(i)
    query = df['Title'][i]
    abstracts, response = get_google_scholar_abstracts(query, num_results=5)
    if response.status_code != 200:
        
    # sleep for 1 second
    time.sleep(1)
    if len(abstracts) > 0:
        df['Abstract'][i] = abstracts[0][1]
    else:
        df['Abstract'][i] = None

# save the dataframe
df.to_csv(file_path, index=False)

In [5]:
# # filter out papers that have corr in Details or have an entry in ArXiv
# papers_that_have_corr = df[df["Details"].str.contains("CoRR", case=False) | df["ArXiv"].str.contains("arxiv", case=False)][["Details", "ArXiv"]]

# # fill out the ArXiv if its empty. Details has CoRR, vol. abs/x.y, ArXiv needs to be filled with https://arxiv.org/abs/x.y
# papers_that_have_corr["ArXiv"] = papers_that_have_corr["ArXiv"].fillna(papers_that_have_corr["Details"].str.extract(r'CoRR, vol. (abs/[\d\.]+)', expand=False).str.replace("abs/", "https://arxiv.org/abs/"))


In [ ]:
# # download papers from Arxiv
# for index, row in papers_that_have_corr.iterrows():
#     id = row["ArXiv"].split("/")[-1]
#     # if id.pdf exists, skip
#     if os.path.exists(f"data/papers/arxiv/{id}.pdf"):
#         continue
#     # arxiv.download(row["ArXiv"], dirpath='./papers/arxiv')
#     paper = next(arxiv.Search(id_list=[id]).results())
#     # Download the PDF to a specified directory with a custom filename.
#     paper.download_pdf(dirpath="data/papers/arxiv", filename=f"{id}.pdf")
